In [378]:
import sys
import numpy as np
from gensim.models.wrappers.fasttext import FastText as ft
from scipy.spatial import distance
import pandas as pd
import MeCab
import json
import warnings

def word2id(bow, word_id):
    for w in bow:
        if w not in word_id:
            word_id[w] = len(word_id)
    return word_id

def compute_tf(sentences, word_id):
    tf = np.zeros([len(sentences), len(word_id)])

    for i in range(len(sentences)):
        for w in sentences[i]:
            tf[i][word_id[w]] += 1
    return tf

def compute_df(sentences, word_id):
    df = np.zeros(len(word_id))

    for i in range(len(sentences)):
        exist = {}
        for w in sentences[i]:
            if w not in exist:
                df[word_id[w]] += 1
                exist[w] = 1
            else:
                continue
    return df

def compute_idf(sentences, word_id):
    idf = np.zeros(len(word_id))
    df = compute_df(sentences, word_id)

    for i in range(len(df)):
        idf[i] = np.log(len(sentences)/df[i]) + 1
    return idf

def compute_tfidf(sentences):
    '''
    INPUT
        sentences: list型。文章のリストで、単語で分かち書きされている。
            (例) [[w11,w12,w13],[w21,w22],[w31,w32,w33,w34]]
    OUTPUT
        tf_idf: np.array型。文章をtf-idf法によってベクトル化したもの。
    '''
    word_id = {}

    for sent in sentences:
        word_id = word2id(sent, word_id)

    tf = compute_tf(sentences, word_id)
    idf = compute_idf(sentences, word_id)

    tf_idf = np.zeros([len(sentences), len(word_id)])

    for i in range(len(sentences)):
        tf_idf[i] = tf[i] * idf

    return tf_idf

def compute_cosine(v1, v2):

    return 1 - distance.cosine(v1, v2)

def sent2vec(bow, model_w):

    vector = np.zeros(100)
    N = len(bow)

    for b in bow:
        try:
            vector += model_w[b]
        except:
            continue

    vector = vector / float(N)

    return vector

def compute_word2vec(sentences):

    model = ft.load_fasttext_format("../Downloads/model")
    
    vector = np.zeros([len(sentences), 100])

    for i in range(len(sentences)):
        vector[i] = sent2vec(sentences[i], model_w)

    return vector

def lexrank(sentences, N, threshold, vectorizer):

    CosineMatrix = np.zeros([N, N])
    degree = np.zeros(N)
    L = np.zeros(N)

    if vectorizer == "tf-idf":
        vector = compute_tfidf(sentences)
    elif vectorizer == "word2vec":
        vector = compute_word2vec(sentences)

    # Computing Adjacency Matrix                                                                                                                                         
    for i in range(N):
        for j in range(N):
            CosineMatrix[i,j] = compute_cosine(vector[i], vector[j])
            if CosineMatrix[i,j] > threshold:
                CosineMatrix[i,j] = 1
                degree[i] += 1
            else:
                CosineMatrix[i,j] = 0

    # Computing LexRank Score                                                                                                                                            
    for i in range(N):
        for j in range(N):
            CosineMatrix[i,j] = CosineMatrix[i,j] / degree[i]

    L = PowerMethod(CosineMatrix, N, err_tol=10e-6)

    return L

def PowerMethod(CosineMatrix, N, err_tol):

    p_old = np.array([1.0/N]*N)
    err = 1

    while err > err_tol:
        err = 1
        p = np.dot(CosineMatrix.T, p_old)
        err = np.linalg.norm(p - p_old)
        p_old = p

    return p

def extractKeyword(text):
    tagger = MeCab.Tagger()
    tagger.parse('')
    node = tagger.parseToNode(text).next
    keywords = []
    while node:
        if node.feature.split(",")[0] == "名詞":
            keywords.append(node.surface)
        node = node.next
    return keywords

def extractKeyword(text,stopword=['']):
    tagger = MeCab.Tagger()
    tagger.parse('')
    node = tagger.parseToNode(text).next
    keywords = []
    hinshi = ['名詞','形容詞','副詞']
    hinshi2 = ['接尾','非自立']
    while node:
        tmp = node.feature.split(',')
        if tmp[0] in hinshi and tmp[1] not in hinshi2 and tmp[-1] not in stopword:
            if tmp[6] == '*' and node.surface not in keywords:
                keywords.append(node.surface)
                #print(keywords[-1], node.feature)
            elif tmp[6] not in keywords:
                keywords.append(tmp[6])
                #print(keywords[-1], node.feature)
            else:
                pass
        node = node.next
    return keywords

In [350]:
def do_lex(report_list, stop, alpha):
    owakachi = [extractKeyword(report_list[x],stop) for x in range(len(report_list))]
    for i in range(len(owakachi)):
        while(1):
            try:
                if owakachi[i] == []:
                    del owakachi[i]
                else:
                    break
            except(IndexError):
                break

    result = lexrank(owakachi,21,alpha,'tf-idf')
    
    output = []
    for k in range(3):
        print(report_list[np.argsort(result)[::-1][k]], np.sort(result)[::-1][k])
        output.append(report_list[np.argsort(result)[::-1][k]])
    return output

In [469]:
def open_json(path, translation):
    report = open(path)
    report = json.load(report)
    report_list = []
    tmp_report = report['selection_reports']
    for i in range(len(tmp_report)):
        tmp_report1 = list(tmp_report[i].values())
        for j in range(len(tmp_report1)):
            tmp_report1[j] = tmp_report1[j].translate(translation)
            try:
                if tmp_report1[j][-1] == '。':
                    tmp_report2 = tmp_report1[j].split('。')
                else:
                    warnings.warn(str(tmp_report1[j]))
                    tmp_report2 = tmp_report1[j].split('。')
            except:
                continue
            for k in range(len(tmp_report2)):
                if tmp_report2[k] != '':
                    report_list.append(tmp_report2[k])
    return report_list, report

In [443]:
def extract_company_name(raw):
    tagger = MeCab.Tagger()
    tagger.parse('')
    node = tagger.parseToNode(raw['name']).next
    return [node.feature.split(',')[6]]

In [415]:
import glob

In [442]:
tagger = MeCab.Tagger()
tagger.parse('')
node = tagger.parseToNode(CA_raw['name']).next

In [460]:
#サイバー
report, raw = open_json('/Users/Shintaro1/Desktop/git/2017_web_biz_4/data/selection_reports/cyber_agent.json',\
                     str.maketrans('．', '。'))
rank = do_lex(report, extract_company_name(raw), 0.1)
output = {}
output['name'] = raw['name']
output['rank'] = rank
f = open('/Users/Shintaro1/Desktop/lex_rank/cyber_agent.json','w')
json.dump(output, f, ensure_ascii=False,indent=2)
f.close()

この会社に入社したいなら、何年目にどのポジションにつくために、まずはこの部署でこの仕事に取り組み、次にこのプランを実施したい、というような人生計画を持っていくべきです 0.0914128252404
ただ、それよりも「一緒に働きたいかどうか」などの基本的な人としての部分が重視されていたと思う 0.076145862243
学生時代どんなことをしてきたのか、そこで感じたことや学んだことがサイバーエージェント・web業界とどうリンクするのかは見られていた気がする 0.0761305834315


In [461]:
#DeNA
report, raw = open_json('/Users/Shintaro1/Desktop/git/2017_web_biz_4/data/selection_reports/dena.json',\
                     str.maketrans('．！', '。。'))
rank = do_lex(report, extract_company_name(raw), 0.1)
output = {}
output['name'] = raw['name']
output['rank'] = rank
f = open('/Users/Shintaro1/Desktop/lex_rank/dena.json','w')
json.dump(output, f, ensure_ascii=False,indent=2)
f.close()

将来に関しては、自分の思い描いている将来がDeNAで達成できるのかどうかを説明できるとなお良いかもしれない 0.0834482153652
圧迫などは全くなく、本当に素の自分がどれだけ会社に合っているかというところを常に意識していたし、意識させられた 0.0696144813613
本選考で見られていた点は●将来に対して自分がどのように有りたいか●内定を出したら、本当に来てくれるのかの2つだと思う 0.0695655846394


/Users/Shintaro1/.pyenv/versions/3.4.3/lib/python3.4/site-packages/ipykernel/__main__.py:13: UserWarning: 論理的思考ができるかどうかと、その人が元々持っている熱量がバランスよく見られていると感じた。ただ、どちらもないとダメと言うわけではなく、どちらか一方が図抜けている学生も歓迎されている印象を受けた。また、選考には非常に時間をかけている会社なので、下手にその場その場で対策しようとするよりは、素の自分を出せるかどうかの方が重要。逆に志望動機などは一切聞かれない。(DeNAを志望していない優秀な学生を心変わりさせるのが目的でもあるので当然といえば当然)


In [462]:
#電通
report, raw = open_json('/Users/Shintaro1/Desktop/git/2017_web_biz_4/data/selection_reports/dentsu.json',\
                     str.maketrans('．！', '。。'))
rank = do_lex(report, extract_company_name(raw), 0.1)
output = {}
output['name'] = raw['name']
output['rank'] = rank
f = open('/Users/Shintaro1/Desktop/lex_rank/dentsu.json','w')
json.dump(output, f, ensure_ascii=False,indent=2)
f.close()

広告業界に行きたい人だけが受かる会社ではないと言われつつも、広告に対する熱意を伝えることは大事だと思います 0.0714789212588
広告についての知識はもちろん、その上で自分は何がやりたいのかを話せる必要があります 0.0714504837233
広告業界というとすごく遠い世界のように感じてしまって、自信を持てない人もいると思いますが、自分のやってきたことを自信を持って話すことができれば可能性は開けてくると思います 0.0595783311426


In [463]:
#ゴールドマン・サックス
report, raw = open_json('/Users/Shintaro1/Desktop/git/2017_web_biz_4/data/selection_reports/goldman_sachs.json',\
                     str.maketrans('．！・', '。。。'))
rank = do_lex(report, extract_company_name(raw), 0.1)
output = {}
output['name'] = raw['name']
output['rank'] = rank
f = open('/Users/Shintaro1/Desktop/lex_rank/goldman_sachs.json','w')
json.dump(output, f, ensure_ascii=False,indent=2)
f.close()

/Users/Shintaro1/.pyenv/versions/3.4.3/lib/python3.4/site-packages/ipykernel/__main__.py:13: UserWarning: 。この仕事に興味を持っているか。誰が相手でもきちんとコミュニケーションをとることができるか（たとえ目上の方にでも臆せず話せるか）。自分の意見をはっきり主張できるか。この部門に合っているかどうか（適性）。ある程度の英語力（完璧とまではいかずとも、”頑張って話そうとしているか”）。ストレス耐性はあるか。どんな場面でも柔軟に対応できるか


オペレーションズの業務に支障のない英語力があるかどうか 0.0626596569701
私が想定していた以上に”人柄”を重視した選考を行っているので、緊張する気持ちも分かりますが面接前に深呼吸をして、普段通りの自分で臨むと良いのではないでしょうか 0.0612244897957
業務内容に関する理解が深まったら、なぜ自分自身がオペレーションズ部門の業務に対する適性があるのかを十分に考えると良いと思います 0.0501217775205


In [465]:
#グーグル
report, raw = open_json('/Users/Shintaro1/Desktop/git/2017_web_biz_4/data/selection_reports/google.json',\
                     str.maketrans('．！', '。。'))
rank = do_lex(report, extract_company_name(raw), 0.05)
output = {}
output['name'] = raw['name']
output['rank'] = rank
f = open('/Users/Shintaro1/Desktop/lex_rank/google.json','w')
json.dump(output, f, ensure_ascii=False,indent=2)
f.close()

今まで受けたIT企業の中でも、すごくプログラミング能力や実践的な思考能力、そして思考プロセスを重視した企業という印象を受けました 0.0877815235269
もちろんある程度正確に問題を解くことは大切ですが、それ以上にどういう手順で問題を解いていくかのほうが重視されている印象がありました 0.068283188616
選考に臨む前に向こうから提示される基礎知識のリストを総復習して、それをコーディングに落とせるようにしとくことが大切だと思います 0.0682655599236


In [473]:
#日本IBM
report, raw = open_json('/Users/Shintaro1/Desktop/git/2017_web_biz_4/data/selection_reports/japan_ibm.json',\
                     str.maketrans('．！・）', '。。。。'))
rank = do_lex(report, extract_company_name(raw), 0.1)
output = {}
output['name'] = raw['name']
output['rank'] = rank
f = open('/Users/Shintaro1/Desktop/lex_rank/japan_ibm.json','w')
json.dump(output, f, ensure_ascii=False,indent=2)
f.close()

/Users/Shintaro1/.pyenv/versions/3.4.3/lib/python3.4/site-packages/ipykernel/__main__.py:14: UserWarning: 【私の思うポイント】。チームで働けるか。一緒に働きたいか。能力（論理的か。。コンサルタントとしての適正【内定を頂いた際に伺ったフィードバック（どの点を見ていたのかの参考に。】GD。ゴールを意識して時間配分をして議論を進めていた。チームの意見を調整して成果を伸ばしていた面接。自分の言葉で発言しており、印象が良かった。人当たりが良く、信頼される人材である。志望動機が面白い
/Users/Shintaro1/.pyenv/versions/3.4.3/lib/python3.4/site-packages/ipykernel/__main__.py:14: UserWarning: この会社は離職率が高く、また営業ということでかなり体力をようす部署であったので、男性と一緒に働けるかなどメンタルやフィジカル的なところが重視されていたと思います。また、営業は顧客とのコミュニケーションがメインの仕事なので、そのコミュニケーション能力もみられていたように思います。質問に的確に答えられるか、明るさや人当たりのよさもみられていたような気がします
/Users/Shintaro1/.pyenv/versions/3.4.3/lib/python3.4/site-packages/ipykernel/__main__.py:14: UserWarning: 内定時のフィードバックによると、グループディスカッションでは自分の意見を伝えることができ、且つ周りの意見を聞きだし受け入れるバランス感覚が評価された。
面接では、相手に伝えるために論理を効果的に用いたことが評価された。（例：ケース面接の前準備で、メリット。デメリットを数字で評価し、それを言葉で説明した。

/Users/Shintaro1/.pyenv/versions/3.4.3/lib/python3.4/site-packages/ipykernel/__main__.py:14: UserWarning: 。論理的に物事を考えられるか。コミュニケーションを積極的にとれるか。他の人の意見を聞けるか。コンサルタント向きな考

自分の経験について、理路整然とはっきりと伝えられ、ビジョンと志望職種が一貫していることが求められるのではないかと思いました 0.0732499187527
そこで、自分に一番合っている職種を選ぶのが非常に重要となるのではないかと思いました 0.0732300376365
GDでは、積極性の有無と、資料を正しく読み込めるかどうか、自分なりにチームに貢献することができるかどうかが重視されていたと思われる 0.0612244897959


In [321]:
#日本IBM
report, raw = open_json('/Users/Shintaro1/Desktop/git/2017_web_biz_4/data/selection_reports/jp_morgan.json',\
                     str.maketrans('．！・）', '。。。。'))
rank = do_lex(report, extract_company_name(raw), 0.1)
output = {}
output['name'] = raw['name']
output['rank'] = rank
f = open('/Users/Shintaro1/Desktop/lex_rank/japan_ibm.json','w')
json.dump(output, f, ensure_ascii=False,indent=2)
f.close()

'就職活動は理不尽なことも多く、落ち込むこともあるかと思います。ただ、周囲の意見、動向に流されることなく、自身の目指す目標に向かって頑張って欲しいと思います。友人、先輩など使えるリソースは全て使って、就職活動を楽しんでいただけたらと思います。道に迷ったときこそ、自分の信じるものに立ち返り、後悔のない選択をしていただけたら幸いです。みなさんの就職活動の成功を期待しています。。'

In [281]:
stop = ['サイバーエージェント','選考']
owakachi = [extractKeyword(report_list[x],stop) for x in range(len(report_list))]
for i in range(len(owakachi)):
    while(1):
        try:
            if owakachi[i] == []:
                del owakachi[i]
            else:
                break
        except(IndexError):
            break

CA = lexrank(owakachi,21,0.1,'tf-idf')
CA_report = report_list
for k in range(5):
    print(report_list[np.argsort(CA)[::-1][k]], np.sort(CA)[::-1][k])

この会社に入社したいなら、何年目にどのポジションにつくために、まずはこの部署でこの仕事に取り組み、次にこのプランを実施したい、というような人生計画を持っていくべきです 0.0914128252404
ただ、それよりも「一緒に働きたいかどうか」などの基本的な人としての部分が重視されていたと思う 0.076145862243
学生時代どんなことをしてきたのか、そこで感じたことや学んだことがサイバーエージェント・web業界とどうリンクするのかは見られていた気がする 0.0761305834315
選考フローが人によって違うこともあるようだが、基本的に他社と比べると多くの選考を通過する必要が今年はあったようだ 0.0609398535163
「サイバーエージェントっぽいかどうか」（それは見た目とかじゃなく精神性も含め）という軸で見られていると思う 0.0609184471618
